In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression

data = fetch_openml('mnist_784', version=1, parser='auto')  # data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"], data["target"]],
                      columns = data["feature_names"] + ["target"])

In [2]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train = dfData[data["feature_names"]].iloc[train_index]
    X_test = dfData[data["feature_names"]].iloc[test_index]
    
    y_train = dfData["target"].iloc[train_index]
    y_test = dfData["target"].iloc[test_index]

In [3]:
from sklearn.model_selection import GridSearchCV

params = [{"penalty":["l1","l2"],"C":[0.1,1,10]}]
#         {"penalty":["elasticnet"],"C":[0.1,1,10, 100],"l1_ratio":[0.1,0.3]}] #"max_iter":[100]

logreg_clf_gscv = GridSearchCV(estimator = LogisticRegression(),
                               param_grid = params,
                               scoring = ["accuracy","roc_auc_ovr_weighted","f1_macro"],
                               refit = "roc_auc_ovr_weighted",  # True
                               cv = 3,  # If our estimator is classifier automatically do stratified CV
                               # n_jobs = -1,  # Num CPUs to use for calculation, -1 means all
                               verbose = 0,  # Output status updates, higher number-> more messages
                               return_train_score = True)  # if false our results won't contain training scores
logreg_clf_gscv.fit(X_train, y_train)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid=[{'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}],
             refit='roc_auc_ovr_weighted', return_train_score=True,
             scoring=['accuracy', 'roc_auc_ovr_weighted', 'f1_macro'])

In [15]:
print(logreg_clf_gscv.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_penalty', 'param_solver', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy', 'split0_test_roc_auc_ovr_weighted', 'split1_test_roc_auc_ovr_weighted', 'split2_test_roc_auc_ovr_weighted', 'mean_test_roc_auc_ovr_weighted', 'std_test_roc_auc_ovr_weighted', 'rank_test_roc_auc_ovr_weighted', 'split0_train_roc_auc_ovr_weighted', 'split1_train_roc_auc_ovr_weighted', 'split2_train_roc_auc_ovr_weighted', 'mean_train_roc_auc_ovr_weighted', 'std_train_roc_auc_ovr_weighted', 'split0_test_f1_macro', 'split1_test_f1_macro', 'split2_test_f1_macro', 'mean_test_f1_macro', 'std_test_f1_macro', 'rank_test_f1_macro', 'split0_train_f1_macro', 'split1_train_f1_macro', 'split2_train_f1_macro', 'mean_t

In [5]:
resultsCVDF = pd.DataFrame(logreg_clf_gscv.cv_results_)
print(resultsCVDF.sort_values("mean_fit_time",ascending=True))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.493883      0.015601         0.000000        0.000000     0.1   
4       0.515670      0.013353         0.000000        0.000000      10   
2       0.586178      0.072537         0.000000        0.000000       1   
1      14.514792      0.282079         2.005798        0.024785     0.1   
3      15.744413      0.133147         2.131112        0.037199       1   
5      15.770614      0.311486         2.240880        0.114965      10   

  param_penalty                       params  split0_test_accuracy  \
0            l1  {'C': 0.1, 'penalty': 'l1'}                   NaN   
4            l1   {'C': 10, 'penalty': 'l1'}                   NaN   
2            l1    {'C': 1, 'penalty': 'l1'}                   NaN   
1            l2  {'C': 0.1, 'penalty': 'l2'}              0.917019   
3            l2    {'C': 1, 'penalty': 'l2'}              0.916484   
5            l2   {'C': 10, 'penalty': 'l2'}          

In [6]:
# best_index_  # only with refit for multi-scoring cases
resultsCVDF.iloc[logreg_clf_gscv.best_index_]

mean_fit_time                                        15.744413
std_fit_time                                          0.133147
mean_score_time                                       2.131112
std_score_time                                        0.037199
param_C                                                      1
param_penalty                                               l2
params                               {'C': 1, 'penalty': 'l2'}
split0_test_accuracy                                  0.916484
split1_test_accuracy                                  0.914287
split2_test_accuracy                                  0.915193
mean_test_accuracy                                    0.915321
std_test_accuracy                                     0.000901
rank_test_accuracy                                           3
split0_train_accuracy                                 0.938285
split1_train_accuracy                                  0.93791
split2_train_accuracy                                 0

In [7]:
print(logreg_clf_gscv.best_estimator_)  # only with refit
logreg_clf_gscv.best_estimator_.predict(X_train)

LogisticRegression(C=1)


array(['7', '0', '6', ..., '5', '4', '5'], dtype=object)

In [8]:
print(logreg_clf_gscv.best_score_)  # only with refit for multi-scoring cases
print(logreg_clf_gscv.best_params_)  # only with refit for multi-scoring cases

0.9924043746498915
{'C': 1, 'penalty': 'l2'}


In [9]:
logreg_clf_gscv.get_params()

{'cv': 3,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(),
 'n_jobs': None,
 'param_grid': [{'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10]}],
 'pre_dispatch': '2*n_jobs',
 'refit': 'roc_auc_ovr_weighted',
 'return_train_score': True,
 'scoring': ['accuracy', 'roc_auc_ovr_weighted', 'f1_macro'],
 'verbose': 0}

In [10]:
logreg_clf_gscv.predict(X_train)  # only when refit is true

array(['7', '0', '6', ..., '5', '4', '5'], dtype=object)

In [11]:
from sklearn.model_selection import RandomizedSearchCV

logreg_clf_rscv = RandomizedSearchCV(estimator = LogisticRegression(),
                                     param_distributions = params,
                                     n_iter = 10,  # num param settings sampled
                                     random_state = None,  # if not none uses this integer as seed
                                     scoring = ["accuracy","roc_auc_ovr_weighted","f1_macro"],
                                     refit = "roc_auc_ovr_weighted",  # True
                                     cv = 3,  # If our estimator is classifier automatically do stratified CV
                                     # n_jobs = -1,  # Num CPUs to use for calculation, -1 means all
                                     verbose = 0,  # Output status updates, higher number-> more messages
                                     return_train_score = True)  # if false our results won't contain training scores
logreg_clf_rscv.fit(X_train, y_train)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

RandomizedSearchCV(cv=3, estimator=LogisticRegression(),
                   param_distributions=[{'C': [0.1, 1, 10],
                                         'penalty': ['l1', 'l2']}],
                   refit='roc_auc_ovr_weighted', return_train_score=True,
                   scoring=['accuracy', 'roc_auc_ovr_weighted', 'f1_macro'])

In [12]:
resultsRSDF = pd.DataFrame(logreg_clf_rscv.cv_results_)
print(resultsRSDF.sort_values("mean_fit_time", ascending=True))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_penalty  \
2       0.494546      0.017001         0.000000        0.000000            l1   
4       0.509363      0.024594         0.000000        0.000000            l1   
0       0.549329      0.054186         0.000000        0.000000            l1   
5      14.318165      0.378823         1.986125        0.014349            l2   
3      15.189989      0.677781         2.055560        0.035031            l2   
1      15.372425      0.660234         2.120950        0.105380            l2   

  param_C                       params  split0_test_accuracy  \
2       1    {'penalty': 'l1', 'C': 1}                   NaN   
4      10   {'penalty': 'l1', 'C': 10}                   NaN   
0     0.1  {'penalty': 'l1', 'C': 0.1}                   NaN   
5      10   {'penalty': 'l2', 'C': 10}              0.916376   
3       1    {'penalty': 'l2', 'C': 1}              0.916484   
1     0.1  {'penalty': 'l2', 'C': 0.1}          

In [13]:
params = [{"penalty": ["l1"], "C": [1], "solver": ["saga"]}]

# params = [{"penalty":["l1"],"C":[0.1,1,10],"solver":["saga"]},
#          {"penalty":["l2"],"C":[0.1,1,10],"solver":["saga"]},
#          {"penalty":["l2"],"C":[0.1,1,10],"solver":["lbfgs"]}]

logreg_clf_gscv = GridSearchCV(estimator = LogisticRegression(),
                               param_grid = params,
                               scoring = ["accuracy","roc_auc_ovr_weighted","f1_macro"],
                               refit = "roc_auc_ovr_weighted",  # True
                               cv = 3,  # If our estimator is classifier automatically do stratified CV
                               # n_jobs=-1,  # Num CPUs to use for calculation, -1 means all
                               verbose = 0,  # Output status updates, higher number-> more messages
                               return_train_score = True)  # if false our results won't contain training scores
logreg_clf_gscv.fit(X_train,y_train)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid=[{'C': [1], 'penalty': ['l1'], 'solver': ['saga']}],
             refit='roc_auc_ovr_weighted', return_train_score=True,
             scoring=['accuracy', 'roc_auc_ovr_weighted', 'f1_macro'])

In [14]:
print(pd.DataFrame(logreg_clf_gscv.cv_results_).iloc[0])

mean_fit_time                                                         814.245986
std_fit_time                                                           16.406915
mean_score_time                                                         2.034508
std_score_time                                                          0.036846
param_C                                                                        1
param_penalty                                                                 l1
param_solver                                                                saga
params                               {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
split0_test_accuracy                                                    0.916644
split1_test_accuracy                                                    0.914127
split2_test_accuracy                                                    0.915729
mean_test_accuracy                                                        0.9155
std_test_accuracy           